Primero, es necesario entrar en https://cloud.google.com. Es necesario registrarse.

Una vez en la consola, navegamos a AI Platform > Notebooks > New Instance.
Indicamos el nombre y dejamos el resto de parámetros por defecto. Pulsamos en Create. 
Con esto hemos podido crear una instancia de un Jupyter Notebook en Google Platform.

Accedemos a Storage > Browser > Create Bucket. Indicamos
 1-Nombre
 2-Location type: Multi-region
 3-Access Control: Fine-grained

Pulsamos Create y tendremos nuestro bucket. Ahora creamos una carpeta en él llamada model.
Subimos a esa carpeta "cats_vs_dogs.h5"



In [ ]:
!gsutil -m cp gs://rhyme-bucket/model/cats_vs_dog

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('cats_vs_dogs.h5')
model.summary()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


<h1>Testing model</h1>

In [ ]:
def display_pred(image_path)
    plt.imshow(plt.imread(image_path))
    
    x = tf.keras.preprocessing.image.load_img(image_path, target_size=(128,128))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
    x = np.expand_dims(x, axis=0)
    
    pred = model.predict(x)[0]
    
    plt.title('Pred {}'.format(classes[int(pred > 0.5)]))
    plt.show()
    
    print(pred)

In [ ]:
images = ['images/cat1.jpg','images/cat2.jpg','images/dog1.jpg','images/dog2.jpg']
display_pred(images[0])
display_pred(images[1])

<h1>Custom prediction class</h1>

In [ ]:
%%writefile prediction.py

import tensorflow as tf
import numpy as np
import os
import base64

MODEL_NAME = 'cats_vs_dogs.h5'
CLASS_NAMES = ['Cat','Dog']

class CatsVsDogsPrediction:
    def __init__(self,model):
        self._model = model
        
    def _preprocess(self,instances):
        num_examples = len(instances)
        x_batch = np.zeros((num_examples,size,size,3))
        for i in range(num_examples):
            x = np.array(bytearray(base64.b64decode(instances[i])))
            x = np.reshape(x,(size,size,3))
            x_batch[i] = x
            return x_batch
        
    def _postprocess(self,preds):
        results = []
        for i, pred in enumerate(preds):
            p = np.squeeze(pred)
            results.append({
                'index' : i,
                'class_name': CLASS_NAMES[(int(p>0.5))],
                'raw_value': '{:.4f}'.format(p)
            })
        return results
            
    def predict(self, instances, **kwargs):
        if 'size' in kwargs:
            size = int(kwargs.get('size'))
        else:
            size = 128
        #Preprocess
        x_batch = self._preprocess(instances,size)
        #Predict
        preds = self._model.predict(x_batch)
        #Postprocess
        results = self._postprocess(pred)
        return results
        
    @classmethod
    def from_path(cls,model_dir):
        model = tf.keras.models.load_model(os.path.join(model_dir,MODEL_NAME))
        return cls(model)

<h1>Setup script</h1>

In [ ]:
%%writefile setup.py
from setuptools import setup
setup{
    name='cats_vs_dogs',
    version='0.0.1',
    include_package_data=False,
    scripts=['prediction.py']
}

In [ ]:
!python3 setup.py sdist --formats=gztar

In [ ]:
!gsutil cp dist/cats_vs_dogs-0.0.1.tar.gz gs://rhyme-bucket/dist/

<h1>Deployment</h1>

Nos dirigimos a AI Platform > Models > Create model. Ingresamos nombre y Create.

Ahora debemos crear una versión en el modelo, mediante la opción Create version. 
Indicamos nombre,
versión 3.7 de Python y en Framework seleccionamos Custom Prediction Routine,
ML runtime version: 1.15
Machine type: Single core CPU
y en la uri del modelo, seleccionamos rhyme-bucket/model.

En prediction class, indicamos prediction.{nombreClasePredictora} y guardamos.

<h1>Predictions</h1>

In [ ]:
from googleapiclient import discovery
from PIL import Image
import os
import base64

In [ ]:
service = discovery.build('ml'.'v1',cache_discovery=False)

In [ ]:
def get_pred_from_model(body, project_name, model_name):
    return service.projects().predict(
    name='projects/{}/models/{}'.format(project_name,model_name),
    body=body  
    )

In [ ]:
project_name = 'rhyme-332122'
model_name = 'cats_vs_dogs'

In [ ]:
instances = []
size = 128

for image in images:
    img = Image.open(image)
    img = img.resize((size,size),Image.ANTIALIAS)
    
    instances.append(
        base64.b64encode(img.tobytes()).decode()
    )
    img.close()

In [ ]:
body = {
    'instances': indstances,
    'size' : size
}

In [ ]:
response = get_pred_from_model(body,project_name,model_name)

In [ ]:
response